In [45]:
import pulp
import time
import numpy as np
import pandas as pd

In [46]:
def lp_Heuristic(filename,sheetname,capacity_num):

#load the data
    data = pd.read_excel(filename, sheetname)
    orders = data[[0,1,2,3,4,5]]
    sum_products = orders.apply(sum, axis=1)
    if capacity_num == 1:
        capacity = data['capacity1'][0]
    else:
        capacity = data['capacity2'][0]
    i_orders = list(orders.index.values)
    i_batch = list(orders.index.values)
    i_order_batch = []
    for order in i_orders:
        for batch in i_batch:
            i_order_batch.append((batch, order))
            
#initialise the model
    model = pulp.LpProblem('The orders to batches problem', pulp.LpMinimize)


# initialise the variables
    x = pulp.LpVariable.dict('x', i_batch, lowBound =0, upBound = 1, cat = pulp.LpInteger) #x_1 = 1 if we use bin 1, 0 else
    y = pulp.LpVariable.dicts('y', i_order_batch ,lowBound = 0, upBound = 1, cat = pulp.LpInteger) #y_(1,1)- 1 if order 1 in batch 1, 0 else 

# create the objective
    model += pulp.lpSum( [x[batch] for batch in i_batch])

# First constraint: For every item, the sum of bins in which it appears must be 1
    for order in i_orders:
        model += pulp.lpSum([y[(batch, order)] for batch in i_batch]) == 1

# Second constraint: desicion variables connection
    for order in i_orders:
        for batch in i_batch:
            model += x[batch] >= y[(batch, order)] 

# third constraint: capacity
    for batch in i_batch:
        model += sum([sum_products[order]*y[(batch, order)] for order in i_orders]) <= capacity2
    
# Solve the optimization.
    start_time = time.time()
    model.solve()

# return results as chromozom
    results = {}
    for key in y.keys():
        if y[key].value()==1:
            results[key[1]]=key[0]
    lst = [None]*20
    for key in results.keys():
        lst[key] = results[key]
    return lst

lp_Heuristic('input.xlsx', '20 orders new format', 1)

[8, 14, 6, 8, 0, 0, 14, 8, 6, 6, 6, 0, 14, 0, 14, 8, 0, 6, 8, 14]